In [ ]:
import os
from pprint import pprint

from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.chat_models import ChatOpenAI
from langchain.chains import RetrievalQA

In [ ]:
!pip show aiohttp
!pip install langchain
!pip install openai
!pip install chromadb
!pip install tiktoken

In [ ]:

with open('key.txt', 'r') as file:
    os.environ["OPENAI_API_KEY"] = file.read().strip()


### prepare data

In [ ]:
data = ''   # load data here using any dataloader

In [ ]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size = 1000, chunk_overlap = 100)
all_splits = text_splitter.split_documents(data)

In [ ]:
vectorstore = Chroma.from_documents(documents=all_splits,
                                    embedding=OpenAIEmbeddings(),
                                    persist_directory='langchain_embeddings')

## set llm

In [ ]:
llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)

In [ ]:
qa_chain = RetrievalQA.from_chain_type(llm,
                                       retriever=vectorstore.as_retriever(),
                                       return_source_documents=True,)

### Test

In [ ]:
question = "Какое метсо занимает Якутия по доступности жилья?"

In [ ]:
result = qa_chain({"query": question})

In [ ]:
print(result['result'])

### tg bot

In [ ]:
!pip install telebot

In [ ]:
import telebot

In [ ]:
bot_hash = ''
bot = telebot.TeleBot(bot_hash)

In [ ]:

@bot.message_handler(content_types=['text'])
def handle_message(message):
    # Проверка на упоминание бота в сообщении
    if '@oyun_bot' in message.text:

        question = message.text.replace('@oyun_bot', '').strip()  # Удаляем упоминание бота из текста вопроса
        bot.send_message(message.chat.id, 'Думаю...')
        result = qa_chain({"query": question + "Не отвечай на вопросы, содержащие нецензурную лексику и прямые оскорбления. Не отвечай на вопросы не связанные с работой органов исполнительной власти Якутии. Проси задать тебе вопросы только по работе государственных органов Якутии."})
        bot.send_message(message.chat.id, result['result'])

while True:
    try:
        bot.polling(none_stop=True)
    except:
        print('Error occurred, retrying...')
        time.sleep(60)
